In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Loading the data and combining training and test dfs.

In [ ]:
df = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

display(df)
display(test)

df_train = df.drop(['Transported'], axis=1)

to_train = pd.concat([df_train, test], axis=0).reset_index().drop('index', axis=1)
display(to_train)

Feature Engineering.

In [ ]:
to_train['Cabin'] = to_train['Cabin'].fillna('Null/5000/Null')
to_train['Cabin Section A'] = [x.split('/')[0] for x in to_train['Cabin']]
to_train['Cabin Section B'] = [x.split('/')[1] for x in to_train['Cabin']]
to_train['Cabin Section C'] = [x.split('/')[2] for x in to_train['Cabin']]

to_train['Passenger Sub ID'] = [x.split('_')[1] for x in to_train['PassengerId']]
to_train['Passenger Group ID'] = [x.split('_')[0] for x in to_train['PassengerId']]

to_train['Name'] = to_train['Name'].fillna('Null Null')
to_train['Family'] = [x.split(' ')[1] for x in to_train['Name']]

to_train = to_train.drop(['Name','Cabin'], axis=1)

display(to_train)

In [ ]:
import math

age_range_cat = []
for age in to_train['Age']:
    if (age>=-100) & (age<=10):
        age_range_cat.append(1)
    if (age>10) & (age<=20):
        age_range_cat.append(2)
    if (age>20) & (age<=30):
        age_range_cat.append(3)
    if (age>30) & (age<=40):
        age_range_cat.append(4)
    if (age>40) & (age<=50):
        age_range_cat.append(5)
    if (age>50) & (age<=60):
        age_range_cat.append(6)
    if (age>60) & (age<=70):
        age_range_cat.append(7)
    if (age>70) & (age<=80):
        age_range_cat.append(8)
    if (age>80) & (age<=90):
        age_range_cat.append(9)
    if (age>90) & (age<=100):
        age_range_cat.append(10)
    if math.isnan(float(age))==True:
        age_range_cat.append(11)

to_train['Age Category'] = age_range_cat
to_train['Age Category'] = to_train['Age Category'].astype('category').cat.codes
to_train['Is Adult'] = [1 if x<18 else 0 for x in to_train['Age']]
to_train = to_train.drop('Age', axis=1)

display(to_train)

In [ ]:
!pip install datawig

Imputation of missing values across columns.

In [ ]:
import datawig

df_train_hpl = to_train[to_train['HomePlanet'].isnull()==False]
df_test_hpl = to_train[to_train['HomePlanet'].isnull()==True]

df_train_cry = to_train[to_train['CryoSleep'].isnull()==False]
df_test_cry = to_train[to_train['CryoSleep'].isnull()==True]

df_train_des = to_train[to_train['Destination'].isnull()==False]
df_test_des = to_train[to_train['Destination'].isnull()==True]

df_train_VIP = to_train[to_train['VIP'].isnull()==False]
df_test_VIP = to_train[to_train['VIP'].isnull()==True]

df_train_cba = to_train[to_train['Cabin Section A'] !='Null']
df_test_cba = to_train[to_train['Cabin Section A'] =='Null']

df_train_cbc = to_train[to_train['Cabin Section C'] !='Null']
df_test_cbc = to_train[to_train['Cabin Section C'] =='Null']

imputer_hpl = datawig.SimpleImputer(
    input_columns= ['Destination','Age Category','RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Cabin Section A', 'Cabin Section B', 'Cabin Section C', 'Passenger Sub ID','Passenger Group ID','Is Adult','Family'], # column(s) containing information about the column we want to impute
    output_column= 'HomePlanet', # the column we'd like to impute values for
    output_path = 'imputer_model' # stores model data and metrics
    )

imputer_cry = datawig.SimpleImputer(
    input_columns= ['HomePlanet','Destination','Age Category', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Cabin Section A', 'Cabin Section B', 'Cabin Section C', 'Passenger Sub ID','Passenger Group ID','Is Adult','Family'], # column(s) containing information about the column we want to impute
    output_column= 'CryoSleep', # the column we'd like to impute values for
    output_path = 'imputer_model' # stores model data and metrics
    )
imputer_des = datawig.SimpleImputer(
    input_columns= ['HomePlanet','Age Category', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Cabin Section A', 'Cabin Section B', 'Cabin Section C', 'Passenger Sub ID','Passenger Group ID','Is Adult','Family'], # column(s) containing information about the column we want to impute
    output_column= 'Destination', # the column we'd like to impute values for
    output_path = 'imputer_model' # stores model data and metrics
    )
imputer_VIP = datawig.SimpleImputer(
    input_columns= ['HomePlanet','Age Category','Destination', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Cabin Section A', 'Cabin Section B', 'Cabin Section C', 'Passenger Sub ID','Passenger Group ID','Is Adult','Family'], # column(s) containing information about the column we want to impute
    output_column= 'VIP', # the column we'd like to impute values for
    output_path = 'imputer_model' # stores model data and metrics
    )

imputer_cba = datawig.SimpleImputer(
    input_columns= ['HomePlanet','Age Category','Destination', 'RoomService', 'ShoppingMall','Spa', 'Cabin Section B', 'Cabin Section C', 'Passenger Sub ID','Passenger Group ID','Is Adult','Family'], # column(s) containing information about the column we want to impute
    output_column= 'Cabin Section A', # the column we'd like to impute values for
    output_path = 'imputer_model' # stores model data and metrics
    )
imputer_cbc = datawig.SimpleImputer(
    input_columns= ['HomePlanet','Age Category','Destination', 'RoomService', 'ShoppingMall','Spa', 'Cabin Section B', 'Passenger Sub ID','Passenger Group ID','Is Adult','Family'], # column(s) containing information about the column we want to impute
    output_column= 'Cabin Section C', # the column we'd like to impute values for
    output_path = 'imputer_model' # stores model data and metrics
    )

In [ ]:
imputer_hpl.fit(train_df=df_train_hpl, num_epochs=10)
imputed_hpl = imputer_hpl.predict(df_test_hpl)

imputer_cry.fit(train_df=df_train_cry, num_epochs=10)
imputed_cry = imputer_cry.predict(df_test_cry)

imputer_des.fit(train_df=df_train_des, num_epochs=10)
imputed_des = imputer_des.predict(df_test_des)

imputer_VIP.fit(train_df=df_train_VIP, num_epochs=10)
imputed_VIP = imputer_VIP.predict(df_test_VIP)

imputer_cba.fit(train_df=df_train_cba, num_epochs=10)
imputed_cba = imputer_cba.predict(df_test_cba)

imputer_cbc.fit(train_df=df_train_cbc, num_epochs=10)
imputed_cbc = imputer_cbc.predict(df_test_cbc)

display(imputed_hpl)

In [ ]:
print(imputed_cry['CryoSleep_imputed_proba'].mean())
print(imputed_des['Destination_imputed_proba'].mean())
print(imputed_VIP['VIP_imputed_proba'].mean())
print(imputed_cba['Cabin Section A_imputed_proba'].mean())
print(imputed_cbc['Cabin Section C_imputed_proba'].mean())

In [ ]:
for x,y in zip(imputed_hpl['HomePlanet_imputed'].index, imputed_hpl['HomePlanet_imputed']):
    to_train.at[x,'HomePlanet'] = y
    
for x,y in zip(imputed_cry['CryoSleep_imputed'].index, imputed_cry['CryoSleep_imputed']):
    to_train.at[x,'CryoSleep'] = y
    
for x,y in zip(imputed_des['Destination_imputed'].index, imputed_des['Destination_imputed']):
    to_train.at[x,'Destination'] = y
    
for x,y in zip(imputed_VIP['VIP_imputed'].index, imputed_VIP['VIP_imputed']):
    to_train.at[x,'VIP'] = y
    
for x,y in zip(imputed_cba['Cabin Section A_imputed'].index, imputed_cba['Cabin Section A_imputed']):
    to_train.at[x,'Cabin Section A'] = y
    
for x,y in zip(imputed_cbc['Cabin Section C_imputed'].index, imputed_cbc['Cabin Section C_imputed']):
    to_train.at[x,'Cabin Section C'] = y
    
to_train['RoomService'] = to_train['RoomService'].fillna(0)
to_train['FoodCourt'] = to_train['FoodCourt'].fillna(0)
to_train['ShoppingMall'] = to_train['ShoppingMall'].fillna(0)
to_train['Spa'] = to_train['Spa'].fillna(0)
to_train['VRDeck'] = to_train['VRDeck'].fillna(0)

Processing for ML.

In [ ]:
to_train['VIP'] = to_train['VIP'].astype('category').cat.codes
to_train['HomePlanet'] = to_train['HomePlanet'].astype('category').cat.codes
to_train['CryoSleep'] = to_train['CryoSleep'].astype('category').cat.codes
to_train['Destination'] = to_train['Destination'].astype('category').cat.codes
to_train['Cabin Section A'] = to_train['Cabin Section A'].astype('category').cat.codes
to_train['Cabin Section C'] = to_train['Cabin Section C'].astype('category').cat.codes
to_train['Passenger Sub ID'] = to_train['Passenger Sub ID'].astype('category').cat.codes
to_train['Passenger Group ID'] = to_train['Passenger Group ID'].astype('category').cat.codes
to_train['Cabin Section B'] = to_train['Cabin Section B'].astype('int')
to_train['Family'] = to_train['Family'].astype('category').cat.codes

to_train = to_train.drop('PassengerId', axis=1)
to_train['Total Spent'] = to_train['RoomService'] + to_train['FoodCourt'] + to_train['ShoppingMall'] + to_train['Spa'] + to_train['VRDeck']
to_train['Essentials Spend'] = to_train['FoodCourt'] + to_train['ShoppingMall']
to_train['Luxuries Spend'] = to_train['Spa'] + to_train['VRDeck'] + to_train['RoomService']

Just a little more feature engineering...

In [ ]:
Group_Size = []
groups = to_train['Passenger Group ID'].unique()
grouped_df = to_train.groupby('Passenger Group ID')

for g in groups:
    Group_Size = len(grouped_df.get_group(g))

group_size = pd.DataFrame()
group_size['Passenger Group ID'] = groups
group_size['Group Size'] = Group_Size

to_train = pd.merge(to_train, group_size, on='Passenger Group ID', how='inner')

Scaling the df.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaled_features = StandardScaler().fit_transform(to_train.values)
scaled_df_train = pd.DataFrame(scaled_features, index=to_train.index, columns=to_train.columns)

df['Transported'] = df['Transported'].astype('category').cat.codes
training_setup = pd.concat([scaled_df_train[:8693], df['Transported']], axis=1)
display(training_setup)

In [ ]:
!pip uninstall pandas -y
!pip install pandas
import pandas as pd

Fitting the CatBoostClassifier and creating a CSV from the predictions of the test data .

In [ ]:
import pandas as pd
from catboost import CatBoostClassifier

cb = CatBoostClassifier(iterations=100000,eval_metric='Accuracy',verbose=5000).fit(training_setup.drop('Transported', axis=1), training_setup['Transported'])
cb_preds = np.array(cb.predict(scaled_df_train[8693:]))
to_sub = [True if int(x)==1 else False for x in cb_preds]

submision = pd.DataFrame(data={'PassengerId':test['PassengerId'], 'Transported':to_sub})


print(submision)